# ReCycleGAN Training in Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://githubtocolab.com/TiagoCAAmorim/dgm-2024.2/blob/main/projetos/ReCycleGAN/src/notebooks/ReCycleGAN_colab.ipynb)

In [16]:
NEXET = '/content/nexet'
OUT_FOLDER = '/content/out'

## Imports

In [1]:
from pathlib import Path
import shutil

directory = Path('/content/dgm-2024.2')
if directory.exists() and directory.is_dir():
    shutil.rmtree(directory)

!git clone -b main https://github.com/TiagoCAAmorim/dgm-2024.2.git

Cloning into 'dgm-2024.2'...
remote: Enumerating objects: 1398, done.
remote: Counting objects: 100% (582/582), done.
remote: Compressing objects: 100% (240/240), done.
remote: Total 1398 (delta 406), reused 421 (delta 340), pack-reused 816 (from 1)
Receiving objects: 100% (1398/1398), 94.39 MiB | 30.62 MiB/s, done.
Resolving deltas: 100% (606/606), done.


In [ ]:
packages = ['pytorch_fid', 'lpips', 'pynvml', 'wandb', 'peft']

for package in packages:
    try:
        __import__(package)
    except ImportError:
        !python -m pip install {package} > /dev/null

In [8]:
from pathlib import Path
import os
os.chdir('/content/dgm-2024.2/projetos/ReCycleGAN/')

from src.dataset import download

NEXET = Path(NEXET)
NEXET.mkdir(parents=True, exist_ok=True)
download.download_and_extract(download.NEXET, NEXET)

## Model Training

In [ ]:
from src.utils.run import init_cyclegan_train, train_cyclegan

In [ ]:
def train(parameters):
    """Trains the CycleGAN model."""
    model, data_loaders = init_cyclegan_train(parameters)
    model = train_cyclegan(model, data_loaders, parameters)

In [ ]:
params = {
    'restart_path': None,
    'parameters_path': None,

    'data_folder': Path(NEXET),
    'csv_type': '_filtered',
    'out_folder': Path(OUT_FOLDER),
    'use_cuda': True,
    'run_wnadb': False,
    'wandb_name': 'Test_Other',
    'print_memory': True,

    "num_epochs" : 100,
    "checkpoint_interval" : 2,
    "n_samples" : None, #None

    'batch_size' : 16,
    'img_height': 256,
    'img_width': 256,

    'channels': 3, #3
    'n_features': 32, #64
    'n_residual_blocks': 5, #9
    'n_downsampling': 2, #2
    'norm_type': 'instance', #'instance' ('batch', 'instance' or 'none')
    'add_skip': True, #False

    'use_replay_buffer': True, #False
    'replay_buffer_size': 50, #50

    'vanilla_loss': False, #True
    'cycle_loss_weight': 10, #10
    'id_loss_weight': 5, #5
    'plp_loss_weight': 1, #5
    'plp_step': 16, #0
    'plp_beta': 0.99, #0.99

    'lr' : 0.0002, #0.0002
    'beta1' : 0.5,  #0.5
    'beta2' : 0.999, #0.999

    'step_size': 10, #20
    'gamma': 0.5, #0.5

    'amp': True, #False
}

train(params)

### Copy Results to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import shutil
source_path = '/content/out'
destination_path = '/content/drive/My Drive/out'
shutil.copytree(source_path, destination_path)